In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from datetime import datetime, timedelta
from pandas import DataFrame

## Import and clean Data

In [3]:
def process(dfurl):

    dfcovid = pd.read_csv(dfurl)
    dfcovid = dfcovid.drop(['UID','iso2', 'iso3','code3', 'FIPS', 'Lat', 'Long_' , 'Country_Region', 'Combined_Key'], axis=1)
    dfcovid.rename(columns = {'Province_State':'State'} , inplace=True)
    dfcovid.rename(columns = {'Admin2':'County'} , inplace=True)
    dfcovid = dfcovid.fillna("NA")
    dfcovid = dfcovid.melt(['State', 'County'], var_name='Date', value_name='value')
    
    return dfcovid

In [4]:
#Import file for confirmed cases per US State and County
urlconfirmed = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
dfconfirmed = process(urlconfirmed)
dfconfirmed.rename(columns = {'value':'Confirmed'} , inplace=True)
dfconfirmed['Date'] = pd.to_datetime(dfconfirmed.Date)
dfconfirmed.head()

,State,County,Date,Confirmed
0,Alabama,Autauga,2020-01-22,0
1,Alabama,Baldwin,2020-01-22,0
2,Alabama,Barbour,2020-01-22,0
3,Alabama,Bibb,2020-01-22,0
4,Alabama,Blount,2020-01-22,0


In [5]:
#Import file for Dead cases per US State and County
urldead = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
dfdead = process(urldead)
dfdead = dfdead[dfdead.Date != 'Population']
dfdead.rename(columns = {'value':'Dead'} , inplace=True)
dfdead['Date'] = pd.to_datetime(dfdead.Date)
dfdead.head()

,State,County,Date,Dead
3340,Alabama,Autauga,2020-01-22,0
3341,Alabama,Baldwin,2020-01-22,0
3342,Alabama,Barbour,2020-01-22,0
3343,Alabama,Bibb,2020-01-22,0
3344,Alabama,Blount,2020-01-22,0


In [21]:
#Merge Confirmed and Dead case numbers on State, County and Date
uscovid_County_lvl = pd.merge(dfconfirmed, dfdead, how='left', left_on=['State', 'County','Date'], right_on = ['State','County', 'Date'])

uscovid_County_lvl

## Get Hospital all beds data

In [8]:
hosp_uti_url = 'https://raw.githubusercontent.com/JieYingWu/COVID-19_US_County-level_Summaries/master/data/Hospitalization_all_locs.csv'
hospital_utilization = pd.read_csv(hosp_uti_url ) 
hospital_utilization = hospital_utilization[hospital_utilization['Attribute'] == 'allbed_mean']
hospital_utilization = hospital_utilization.melt(['State', 'Attribute'], var_name='Date', value_name='allbed_mean')
hospital_utilization['Date'] = pd.to_datetime(hospital_utilization.Date)
hospital_utilization = hospital_utilization.drop(['Attribute'], axis=1)
hospital_utilization

,State,Date,allbed_mean
0,Alabama,2019-12-13,0.0
1,Alaska,2019-12-13,0.0
2,Arizona,2019-12-13,0.0
3,Arkansas,2019-12-13,0.0
4,California,2019-12-13,0.0
...,...,...,...
13051,Virginia,2020-08-24,0.0
13052,Washington,2020-08-24,0.0
13053,West Virginia,2020-08-24,0.0
13054,Wisconsin,2020-08-24,0.0


In [23]:
uscovid_statelevel = uscovid_County_lvl.drop('County', axis =1)
uscovid_statelevel = pd.DataFrame(uscovid_statelevel.groupby(['State','Date']).agg({'Confirmed': 'sum', 'Dead': 'sum'})).reset_index()


In [25]:
uscovid_statelevel

,State,Date,Confirmed,Dead
0,Alabama,2020-01-22,0,0
1,Alabama,2020-01-23,0,0
2,Alabama,2020-01-24,0,0
3,Alabama,2020-01-25,0,0
4,Alabama,2020-01-26,0,0
...,...,...,...,...
15365,Wyoming,2020-10-08,7092,54
15366,Wyoming,2020-10-09,7335,54
15367,Wyoming,2020-10-10,7455,54
15368,Wyoming,2020-10-11,7611,54


In [26]:
uscovid_statelevel = pd.merge(uscovid_statelevel, hospital_utilization, how='left')

In [27]:
print(uscovid_statelevel.sort_values('allbed_mean', ascending=True))

         State       Date  Confirmed  Dead  allbed_mean
0      Alabama 2020-01-22          0     0          0.0
8062   Montana 2020-05-13        462    16          0.0
8061   Montana 2020-05-12        461    16          0.0
8060   Montana 2020-05-11        459    16          0.0
8059   Montana 2020-05-10        458    16          0.0
...        ...        ...        ...   ...          ...
15365  Wyoming 2020-10-08       7092    54          NaN
15366  Wyoming 2020-10-09       7335    54          NaN
15367  Wyoming 2020-10-10       7455    54          NaN
15368  Wyoming 2020-10-11       7611    54          NaN
15369  Wyoming 2020-10-12       7802    54          NaN

[15370 rows x 5 columns]


## Join Census Data

In [13]:
#Import Census Data
census = pd.read_csv("C:/Users/ysman/Documents/springboard/CAPSTONE 2/Covid/covidpython/nst-est2019-alldata.csv", header=0)
census_col_list = ['NAME' , 'POPESTIMATE2019']
census2019 = census[census_col_list ]

census2019.rename(columns = {'POPESTIMATE2019':'Population'}, inplace = True)
census2019.rename(columns =  {'NAME':'State'}, inplace = True)
census2019.head()

,State,Population
0,Alabama,4903185
1,Alaska,731545
2,Arizona,7278717
3,Arkansas,3017804
4,California,39512223


In [28]:
# Merge Census data into main df
uscovid_statelevel = pd.merge(uscovid_statelevel, census2019, how='left')
uscovid_statelevel.head()

,State,Date,Confirmed,Dead,allbed_mean,Population
0,Alabama,2020-01-22,0,0,0.0,4903185.0
1,Alabama,2020-01-23,0,0,0.0,4903185.0
2,Alabama,2020-01-24,0,0,0.0,4903185.0
3,Alabama,2020-01-25,0,0,0.0,4903185.0
4,Alabama,2020-01-26,0,0,0.0,4903185.0


## Create columns for Total Confirmed and Total Dead

In [29]:
TotalConfirmed = uscovid_statelevel.groupby(['State'])['Confirmed'].sum()
Totaldf = TotalConfirmed.to_frame().reset_index()
Totaldf.rename(columns =  {'Confirmed':'Total_Confirmed'}, inplace = True)
Totaldf.head()

,State,Total_Confirmed
0,Alabama,12736577
1,Alaska,528291
2,American Samoa,0
3,Arizona,20890998
4,Arkansas,6450699


In [30]:
TotalDead = uscovid_statelevel.groupby(['State'])['Dead'].sum()
Totaldf2 = TotalDead.to_frame().reset_index()
Totaldf2.rename(columns =  {'Dead':'Total_Dead'}, inplace = True)

In [31]:
#Merge Total Confirmed and TotalDead in uscovid file
uscovid_statelevel = pd.merge(uscovid_statelevel, Totaldf, how='left')

In [32]:
uscovid_statelevel = pd.merge(uscovid_statelevel, Totaldf2, how='left')

In [33]:
uscovid_statelevel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15370 entries, 0 to 15369
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   State            15370 non-null  object        
 1   Date             15370 non-null  datetime64[ns]
 2   Confirmed        15370 non-null  int64         
 3   Dead             15370 non-null  int64         
 4   allbed_mean      11016 non-null  float64       
 5   Population       13780 non-null  float64       
 6   Total_Confirmed  15370 non-null  int64         
 7   Total_Dead       15370 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 1.1+ MB


In [34]:
uscovid_statelevel

,State,Date,Confirmed,Dead,allbed_mean,Population,Total_Confirmed,Total_Dead
0,Alabama,2020-01-22,0,0,0.0,4903185.0,12736577,242163
1,Alabama,2020-01-23,0,0,0.0,4903185.0,12736577,242163
2,Alabama,2020-01-24,0,0,0.0,4903185.0,12736577,242163
3,Alabama,2020-01-25,0,0,0.0,4903185.0,12736577,242163
4,Alabama,2020-01-26,0,0,0.0,4903185.0,12736577,242163
...,...,...,...,...,...,...,...,...
15365,Wyoming,2020-10-08,7092,54,NaN,578759.0,456267,4636
15366,Wyoming,2020-10-09,7335,54,NaN,578759.0,456267,4636
15367,Wyoming,2020-10-10,7455,54,NaN,578759.0,456267,4636
15368,Wyoming,2020-10-11,7611,54,NaN,578759.0,456267,4636


### Exporting CSV

In [36]:
uscovid_County_lvl.to_csv('C:/Users/ysman/Documents/springboard/1 Google Mobility and Covid/python df exports/Covid_Countylevel.csv', sep=',',index=True)
uscovid_statelevel.to_csv('C:/Users/ysman/Documents/springboard/1 Google Mobility and Covid/python df exports/Covid_Statelevel.csv', sep=',',index=True)